In [1]:
#to export a notebook to html use:
# >ipython nbconvert --to html Process-GitHub-Repos.ipynb

from pandas import DataFrame, Series
import pandas as pd
import matplotlib.pyplot as plt
import string
from datetime import datetime, date, time

import re
%pylab inline
import numpy as np
import scipy as sc
import scipy.stats as stats
from scipy.optimize import leastsq
from graphviz import Digraph
import operator

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [56]:
from collections import Counter

data_file = './datasets/journeys.csv'
file = open(data_file, 'r')
df_f = DataFrame.from_csv(data_file)
df = df_f.loc[:,('trans_id','topic_intent_eng','channel','action_eng','cluster')]
df['journey_id'] = df.index

In [3]:
df['channel_gen'] = map(lambda c: 'live' if c in ['f2f','phone'] else 'self', df['channel'] )

In [86]:
df1 = df.loc[:,('trans_id','topic_intent_eng','channel','action_eng','cluster')].reset_index()
df2 = df1.copy()

df1 = df1.rename(columns={'action_eng':'from_action'})
df2 = df2.rename(columns={'action_eng':'to_action'})

df2['trans_id'] = df2['trans_id'] - 1
# df1.loc[df1.trans_id == 1, 'from_action'] = 'start'

# df1['jt_id'] = df1.jorney_id.map(str) + '_' +  df1.trans_id.map(str)
# df2['jt_id'] = df2.jorney_id.map(str) + '_' +  df2.trans_id.map(str)

In [271]:
df_trans = pd.merge(df1, df2, how='outer', on=['jorney_id','trans_id'], sort=['jorney_id','trans_id'])

df_trans.from_action.fillna('start', inplace=True)
df_trans.to_action.fillna('end', inplace=True)

df_trans.fillna('',inplace=True)
df_trans['cluster'] = np.where(df_trans.cluster_x == '', df_trans.cluster_y, df_trans.cluster_x)

df_trans = df_trans.loc[:,('jorney_id','trans_id', 'cluster' ,'from_action','to_action')]
df_trans

,jorney_id,trans_id,cluster,from_action,to_action
0,1,0,1,start,purchase
1,1,1,1,purchase,end
2,2,0,2,start,purchase
3,2,1,2,purchase,end
4,3,0,1,start,purchase
5,3,1,1,purchase,end
6,4,0,2,start,request
7,4,1,2,request,collect at branch
8,4,2,2,collect at branch,end
9,5,0,1,start,proposal


In [232]:
df_trans = df_trans[df_trans.to_action != 'end']
df_action_action_count = DataFrame(df_trans.groupby(['cluster','from_action','to_action']).count().loc[:,('trans_id')]).rename(columns={'trans_id':'count'})
df_action_action_cpt = df_action_action_count.groupby(level=[0,1]).transform(lambda x: x/x.sum()).rename(columns={'count':'prob'})
# df_trans.groupby(['cluster','from_action','to_action']).transform(lambda x: (x-x.mean())/x.std())
df_action_action_count

count 
 
 
 cluster 
 from_action 
 to_action 
 
 
 
 
 
 1 
 inquiry 
 collect at branch 
 2 
 
 
 inquiry 
 6 
 
 
 response 
 2 
 
 
 proposal 
 request 
 1 
 
 
 request 
 inquiry 
 3 
 
 
 response 
 1 
 
 
 response 
 sign documents 
 1 
 
 
 talk with manager 
 1 
 
 
 start 
 change details 
 1 
 
 
 collect at branch 
 1 
 
 
 inquiry 
 3 
 
 
 proposal 
 1 
 
 
 purchase 
 2 
 
 
 request 
 4 
 
 
 withdrawal 
 1 
 
 
 2 
 inquiry 
 collect at branch 
 2 
 
 
 inquiry 
 3 
 
 
 message 
 notification 
 1 
 
 
 notification 
 talk with manager 
 1 
 
 
 request 
 collect at branch 
 1 
 
 
 inquiry 
 1 
 
 
 start 
 change details 
 2 
 
 
 inquiry 
 4 
 
 
 message 
 1 
 
 
 purchase 
 1 
 
 
 request 
 3 
 
 
 talk with manager 
 1 
 
 
 withdrawal 
 1

In [427]:
distinct_actions = df.action_eng.unique().tolist()+['start']
clusters = df.cluster.unique()
from_to_actions = []
for c in clusters:
    for f in distinct_actions:
        for t in distinct_actions:
            from_to_actions.append((c,f,t,1))
        
df_action_action_smooth_base = DataFrame(from_to_actions, columns=['cluster','from_action', 'to_action', 'count'])
df_action_action_smooth_base

,cluster,from_action,to_action,count
0,1,purchase,purchase,1
1,1,purchase,request,1
2,1,purchase,collect at branch,1
3,1,purchase,proposal,1
4,1,purchase,inquiry,1
5,1,purchase,change details,1
6,1,purchase,withdrawal,1
7,1,purchase,response,1
8,1,purchase,talk with manager,1
9,1,purchase,message,1


In [413]:
df_trans

,jorney_id,trans_id,cluster,from_action,to_action
0,1,0,1,start,purchase
1,1,1,1,purchase,end
2,2,0,2,start,purchase
3,2,1,2,purchase,end
4,3,0,1,start,purchase
5,3,1,1,purchase,end
6,4,0,2,start,request
7,4,1,2,request,collect at branch
8,4,2,2,collect at branch,end
9,5,0,1,start,proposal


In [412]:
df_action_action_count_smothed = pd.merge(df_action_action_smooth_base, df_action_action_count.reset_index(), how='left', on=['cluster', 'from_action', 'to_action'], suffixes=['','_y'])
df_action_action_count_smothed.count_y.fillna(0, inplace=True)
df_action_action_count_smothed['count'] = df_action_action_count_smothed['count'] + df_action_action_count_smothed['count_y']*100
df_action_action_count_smothed.drop('count_y', inplace=True, axis=1)
df_action_action_count_smothed

,cluster,from_action,to_action,count
0,1,purchase,purchase,1
1,1,purchase,request,1
2,1,purchase,collect at branch,1
3,1,purchase,proposal,1
4,1,purchase,inquiry,1
5,1,purchase,change details,1
6,1,purchase,withdrawal,1
7,1,purchase,response,1
8,1,purchase,talk with manager,1
9,1,purchase,message,1


In [378]:
actions =Series(1, index=df.action_eng.unique())
actions = DataFrame(actions, columns=['norm'])
actions['action'] = actions.index

In [314]:
df_action_action_count_for_norm = df_action_action_count.reset_index()
df_action_action_count_for_norm['count'] = df_action_action_count_for_norm['count']*100

def tr(g):    
    zz = pd.merge(actions, g, left_on='action',right_on='to_action', how='left', sort=['action', 'to_action'])
    zz['count'].fillna(0, inplace=True)
    zz['count'] = zz['count']+zz['norm']
    return zz

df_action_action_norm_count = df_action_action_count_for_norm.groupby(['cluster','from_action']).apply(tr)


df_action_action_norm_count.drop(['norm', 'from_action', 'cluster', 'to_action'], inplace=True, axis=1)
df_action_action_norm_count.rename(columns={'action':'to_action'}, inplace=True)
df_action_action_norm_count.reset_index(inplace=True)
df_action_action_norm_count.drop('level_2',inplace=True, axis=1)
df_action_action_norm_count.set_index(['cluster', 'from_action', 'to_action'], inplace=True)
# df_action_action_norm_count
gr_action_action_norm_cpt = df_action_action_norm_count.groupby(level=[0,1]).transform(lambda x: x/x.sum()).rename(columns={'count':'prob'})
gr_action_action_norm_cpt

prob 
 
 
 cluster 
 from_action 
 to_action 
 
 
 
 
 
 1 
 inquiry 
 change details 
 0.000988 
 
 
 collect at branch 
 0.198617 
 
 
 inquiry 
 0.593874 
 
 
 message 
 0.000988 
 
 
 notification 
 0.000988 
 
 
 proposal 
 0.000988 
 
 
 purchase 
 0.000988 
 
 
 request 
 0.000988 
 
 
 response 
 0.198617 
 
 
 sign documents 
 0.000988 
 
 
 talk with manager 
 0.000988 
 
 
 withdrawal 
 0.000988 
 
 
 proposal 
 change details 
 0.008929 
 
 
 collect at branch 
 0.008929 
 
 
 inquiry 
 0.008929 
 
 
 message 
 0.008929 
 
 
 notification 
 0.008929 
 
 
 proposal 
 0.008929 
 
 
 purchase 
 0.008929 
 
 
 request 
 0.901786 
 
 
 response 
 0.008929 
 
 
 sign documents 
 0.008929 
 
 
 talk with manager 
 0.008929 
 
 
 withdrawal 
 0.008929 
 
 
 request 
 change details 
 0.002427 
 
 
 collect at branch 
 0.002427 
 
 
 inquiry 
 0.730583 
 
 
 message 
 0.002427 
 
 
 notification 
 0.002427 
 
 
 proposal 
 0.002427 
 
 
 ... 
 ... 
 ... 
 ... 
 
 
 2 
 notification 
 purchase 
 0.008929 
 
 
 request 
 0.008929 
 
 
 response 
 0.008929 
 
 
 sign documents 
 0.008929 
 
 
 talk with manager 
 0.901786 
 
 
 withdrawal 
 0.008929 
 
 
 request 
 change details 
 0.004717 
 
 
 collect at branch 
 0.476415 
 
 
 inquiry 
 0.476415 
 
 
 message 
 0.004717 
 
 
 notification 
 0.004717 
 
 
 proposal 
 0.004717 
 
 
 purchase 
 0.004717 
 
 
 request 
 0.004717 
 
 
 response 
 0.004717 
 
 
 sign documents 
 0.004717 
 
 
 talk with manager 
 0.004717 
 
 
 withdrawal 
 0.004717 
 
 
 start 
 change details 
 0.153201 
 
 
 collect at branch 
 0.000762 
 
 
 inquiry 
 0.305640 
 
 
 message 
 0.076982 
 
 
 notification 
 0.000762 
 
 
 proposal 
 0.000762 
 
 
 purchase 
 0.076982 
 
 
 request 
 0.229421 
 
 
 response 
 0.000762 
 
 
 sign documents 
 0.000762 
 
 
 talk with manager 
 0.076982 
 
 
 withdrawal 
 0.076982 
 
 
 
 120 rows × 1 columns

In [317]:
df_action_action_count

count 
 
 
 cluster 
 from_action 
 to_action 
 
 
 
 
 
 1 
 inquiry 
 collect at branch 
 2 
 
 
 inquiry 
 6 
 
 
 response 
 2 
 
 
 proposal 
 request 
 1 
 
 
 request 
 inquiry 
 3 
 
 
 response 
 1 
 
 
 response 
 sign documents 
 1 
 
 
 talk with manager 
 1 
 
 
 start 
 change details 
 1 
 
 
 collect at branch 
 1 
 
 
 inquiry 
 3 
 
 
 proposal 
 1 
 
 
 purchase 
 2 
 
 
 request 
 4 
 
 
 withdrawal 
 1 
 
 
 2 
 inquiry 
 collect at branch 
 2 
 
 
 inquiry 
 3 
 
 
 message 
 notification 
 1 
 
 
 notification 
 talk with manager 
 1 
 
 
 request 
 collect at branch 
 1 
 
 
 inquiry 
 1 
 
 
 start 
 change details 
 2 
 
 
 inquiry 
 4 
 
 
 message 
 1 
 
 
 purchase 
 1 
 
 
 request 
 3 
 
 
 talk with manager 
 1 
 
 
 withdrawal 
 1

In [353]:
df_action_action_count_for_norm = df_action_action_count.reset_index()

df_action_action_norm_count = df_action_action_count_for_norm.groupby(['cluster']) \
    .apply(lambda g: pd.merge(actions, g, left_on='action',right_on='from_action', how='left', sort=['action', 'from_action'])  )
df_action_action_norm_count.drop(['norm', 'from_action', 'cluster', 'from_action'], inplace=True, axis=1)
df_action_action_norm_count.rename(columns={'action':'from_action'}, inplace=True)
df_action_action_norm_count.reset_index(inplace=True)
df_action_action_norm_count.drop('level_1',inplace=True, axis=1)


# df_action_action_norm_count++++++++++++++++++++++++++++++

df_action_action_count_for_norm = df_action_action_norm_count

df_action_action_count_for_norm['count'] = df_action_action_count_for_norm['count']*100

def tr(g):    
    zz = pd.merge(actions, g, left_on='action',right_on='to_action', how='left', sort=['action', 'to_action'])
    zz['count'].fillna(0, inplace=True)
    zz['count'] = zz['count']+zz['norm']
    return zz

df_action_action_norm_count = df_action_action_count_for_norm.groupby(['cluster','from_action']).apply(tr)


df_action_action_norm_count.drop(['norm', 'from_action', 'cluster', 'to_action'], inplace=True, axis=1)
df_action_action_norm_count.rename(columns={'action':'to_action'}, inplace=True)
df_action_action_norm_count.reset_index(inplace=True)
df_action_action_norm_count.drop('level_2',inplace=True, axis=1)
df_action_action_norm_count.set_index(['cluster', 'from_action', 'to_action'], inplace=True)
# df_action_action_norm_count
gr_action_action_norm_cpt = df_action_action_norm_count.groupby(level=[0,1]).transform(lambda x: x/x.sum()).rename(columns={'count':'prob'})
# print gr_action_action_norm_cpt.ix[1]

def print_full(x):
    pd.set_option('display.max_rows', len(x))
    print(x)
    pd.reset_option('display.max_rows')

print_full(gr_action_action_norm_cpt)

                                                 prob
cluster from_action       to_action                  
1       change details    change details     0.083333
                          collect at branch  0.083333
                          inquiry            0.083333
                          message            0.083333
                          notification       0.083333
                          proposal           0.083333
                          purchase           0.083333
                          request            0.083333
                          response           0.083333
                          sign documents     0.083333
                          talk with manager  0.083333
                          withdrawal         0.083333
        collect at branch change details     0.083333
                          collect at branch  0.083333
                          inquiry            0.083333
                          message            0.083333
                          no

In [180]:
df_trans_gr = df_action_action_cpt.reset_index().groupby(['cluster'])
# df_trans_gr = df_trans.groupby(['cluster'])

# def f(gr):
#     return Series(gr)
#     print type(gr)
#     pd.merge(actions, DataFrame(gr), how='left', left_on='action', right_on='action_from')
    
# df_trans_gr.transform(f)
l = []
for cl_g in df_trans_gr:
    join_from = pd.merge(actions, cl_g[1], left_on='action',right_on='from_action', how='left', sort=['action', 'from_action'])
    gr_join_from = join_from.groupby('from_action')
    for fr_g in gr_join_from:
        join_fr_to = pd.merge(actions, fr_g[1], left_on='action',right_on='to_action', how='left', sort=['action', 'to_action'])
        l.append(join_fr_to)
  
len(l)
    
#     l.append(g.groupby(['from_action']))

# qq = pd.merge(actions, l[0][1], left_on='action',right_on='from_action', how='left', sort=['action', 'from_action'])
# qq.from_action = qq.action

# pd.merge(action, qq, left_on='action',right_on='to_action', how='left', sort=['action', 'from_action'])

8

In [43]:
df_action_channel_count = DataFrame(df.groupby(['cluster','action_eng','channel']).count().loc[:,('trans_id')]).rename(columns={'trans_id':'count'})
df_action_chanel_cpt = df_action_channel_count.groupby(level=[0,1]).transform(lambda x: x/x.sum()).rename(columns={'count':'prob'})

df_action_chanel_cpt

prob 
 
 
 cluster 
 action_eng 
 channel 
 
 
 
 
 
 1 
 change details 
 web 
 1.000000 
 
 
 collect at branch 
 f2f 
 1.000000 
 
 
 inquiry 
 phone 
 0.833333 
 
 
 web 
 0.166667 
 
 
 proposal 
 phone 
 1.000000 
 
 
 purchase 
 phone 
 0.500000 
 
 
 web 
 0.500000 
 
 
 request 
 phone 
 0.600000 
 
 
 web 
 0.400000 
 
 
 response 
 phone 
 1.000000 
 
 
 sign documents 
 f2f 
 1.000000 
 
 
 talk with manager 
 f2f 
 1.000000 
 
 
 withdrawal 
 ivr 
 1.000000 
 
 
 2 
 change details 
 f2f 
 0.500000 
 
 
 phone 
 0.500000 
 
 
 collect at branch 
 f2f 
 1.000000 
 
 
 inquiry 
 f2f 
 0.125000 
 
 
 ivr 
 0.250000 
 
 
 phone 
 0.500000 
 
 
 web 
 0.125000 
 
 
 message 
 post 
 1.000000 
 
 
 notification 
 f2f 
 1.000000 
 
 
 purchase 
 f2f 
 1.000000 
 
 
 request 
 f2f 
 0.666667 
 
 
 phone 
 0.333333 
 
 
 talk with manager 
 f2f 
 1.000000 
 
 
 withdrawal 
 f2f 
 1.000000

# OLD FASHION

In [418]:
df_journeys_group = df.groupby(['journey_id'])
actions_by_journey = df_journeys_group['action_eng'].apply(lambda x: ','.join(x)).reset_index()
channels_by_journey = df_journeys_group['channel'].apply(lambda x: ','.join(x)).reset_index()
channels_gen_by_journey = df_journeys_group['channel_gen'].apply(lambda x: ','.join(x)).reset_index()

In [60]:
df_journeys = actions_by_journey.join(channels_by_journey['channel']).join(channels_gen_by_journey['channel_gen'])
df_journeys

,journey_id,action_eng,channel,channel_gen
0,1,purchase,web,self
1,2,purchase,f2f,live
2,3,purchase,phone,live
3,4,"request,collect at branch","phone,f2f","live,live"
4,5,"proposal,request,inquiry,collect at branch","phone,phone,phone,f2f","live,live,live,live"
5,6,"request,inquiry,inquiry,inquiry,collect at branch","web,phone,phone,phone,f2f","self,live,live,live,live"
6,7,"request,inquiry,collect at branch","f2f,phone,f2f","live,live,live"
7,8,"inquiry,collect at branch","phone,f2f","live,live"
8,9,collect at branch,f2f,live
9,10,inquiry,web,self


In [140]:
def create_counter(df, id_column,column):
    state_cnt = Counter()
    # overall_state_trans = Counter()
#     list_state_trans = []
    state_trans_cnt = Counter()
    state_start_cnt = Counter()
    
    for index,line in df.loc[:,(id_column,column)].iterrows():       
        traj1 = ['start']+line[column].split(',')

        for state in traj1:
            state_cnt[state]+=1

#         traj2 = ['start']+line[column].split(',')
        
        for prevstate,state in zip(traj1[:-1],traj1[1:]):
    #         state_trans_cnt[prevstate+">"+state]+=1
            state_trans_cnt[(prevstate,state)]+=1
            state_start_cnt[prevstate]+=1
    #         overall_state_trans[prevstate+">"+state]+=1

#         list_state_trans.append(state_trans_cnt)
        
    
    return state_cnt, state_start_cnt, state_trans_cnt

In [141]:
channel_state_cnt, channel_state_start_cnt, channel_state_trans_cnt = create_counter(df_journeys, 'journey_id', 'channel_gen')
action_state_cnt, action_state_start_cnt, action_state_trans_cnt = create_counter(df_journeys, 'journey_id', 'action_eng')

In [195]:
print channel_state_cnt
print
print channel_state_trans_cnt
print '='*50
print action_state_cnt
print 
print action_state_trans_cnt

Counter({'live': 41, 'start': 26, 'self': 11})

Counter({('live', 'live'): 21, ('start', 'live'): 15, ('start', 'self'): 11, ('self', 'live'): 5})
Counter({'start': 26, 'inquiry': 20, 'request': 8, 'collect at branch': 6, 'purchase': 3, 'change details': 3, 'talk with manager': 3, 'response': 3, 'withdrawal': 2, 'notification': 1, 'message': 1, 'proposal': 1, 'sign documents': 1})

Counter({('inquiry', 'inquiry'): 9, ('start', 'inquiry'): 7, ('start', 'request'): 7, ('request', 'inquiry'): 4, ('inquiry', 'collect at branch'): 4, ('start', 'purchase'): 3, ('start', 'change details'): 3, ('inquiry', 'response'): 2, ('start', 'withdrawal'): 2, ('start', 'talk with manager'): 1, ('request', 'response'): 1, ('response', 'talk with manager'): 1, ('start', 'collect at branch'): 1, ('message', 'notification'): 1, ('start', 'proposal'): 1, ('request', 'collect at branch'): 1, ('start', 'message'): 1, ('notification', 'talk with manager'): 1, ('proposal', 'request'): 1, ('response', 'sign docume

In [166]:
def get_cpt(transitions_counts):
    row_count = Counter()
    for t,c in transitions_counts.iteritems():
        row_count[t[0]] += c
        
    cpt = {}
    for t,c in transitions_counts.iteritems():
        cpt[t] = float(c)/float(row_count[t[0]])
        
    return row_count, cpt

In [305]:
action_from_count, action_action_cpt =get_cpt(action_state_trans_cnt)
action_action_cpt

{('inquiry', 'collect at branch'): 0.26666666666666666,
 ('inquiry', 'inquiry'): 0.6,
 ('inquiry', 'response'): 0.13333333333333333,
 ('message', 'notification'): 1.0,
 ('notification', 'talk with manager'): 1.0,
 ('proposal', 'request'): 1.0,
 ('request', 'collect at branch'): 0.16666666666666666,
 ('request', 'inquiry'): 0.6666666666666666,
 ('request', 'response'): 0.16666666666666666,
 ('response', 'sign documents'): 0.5,
 ('response', 'talk with manager'): 0.5,
 ('start', 'change details'): 0.11538461538461539,
 ('start', 'collect at branch'): 0.038461538461538464,
 ('start', 'inquiry'): 0.2692307692307692,
 ('start', 'message'): 0.038461538461538464,
 ('start', 'proposal'): 0.038461538461538464,
 ('start', 'purchase'): 0.11538461538461539,
 ('start', 'request'): 0.2692307692307692,
 ('start', 'talk with manager'): 0.038461538461538464,
 ('start', 'withdrawal'): 0.07692307692307693}

In [206]:
actions_count

Counter({'start': 26, 'inquiry': 15, 'request': 6, 'response': 2, 'notification': 1, 'message': 1, 'proposal': 1})